In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
!pip install requests --upgrade --quiet --use-feature=2020-resolver
!pip install beautifulsoup4 --upgrade --quiet

In [4]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def item_retriever(page):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    response = requests.get(page, headers=headers)
    doc = BeautifulSoup(response.text,'html.parser')
    items = doc.find_all('a', {'class':"s-item__link"})
    return items

In [ ]:
item_titles = []
item_links = []
condition = [] 
price_usd = []
item_specs = []
#page_data = pd.DataFrame(list(zip(item_titles, item_links)),columns =['Name', 'Links'])

In [ ]:
item_titles = []
item_links = []

for i in range(1,60):
    print("Page No:",i)
    page = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=ipad&_sacat=0&_pgn=" + str(i)
    items = item_retriever(page)

    for item in items:
        item_titles.append(item.text)
        item_links.append(item.get('href'))
        #print(item.get('href'))

In [ ]:
my_data = pd.DataFrame(list(zip(item_titles, item_links)),
               columns =['Name', 'Links'])

In [ ]:
my_data.to_csv("link_ebay_data.csv")

In [ ]:
sum(my_data.duplicated())

In [ ]:
new_data = my_data.drop_duplicates()

In [ ]:
new_data = new_data.reset_index().drop(["index"], axis=1)

In [ ]:
new_data.to_csv("clean_ebaylinks_data.csv")

In [ ]:
len(new_data)

In [ ]:
condition = [] 
price_usd = []
item_specs = []
for p in range(len(new_data.Links)):
    page_link = new_data.Links[p]
    
    if p%10 ==0:
        print("now in page:",p)
    
    response = requests.get(page_link)
    soup = BeautifulSoup(response.text,'html.parser')
    
    #Creating a list of condition values
    try:
        condition.append(soup.find_all('div',class_ = "u-flL condText")[0].text)
    except:
        condition.append(None)
    
    #Creating a list of price values
    try:
        price_usd.append(float(soup.find_all('span',class_ = "notranslate")[0].get('content')))
    except:
        price_usd.append(None)

    #Creating a list of itemspecifics values
    try:
        item_specifics = soup.find_all('div',class_ = "ux-layout-section__item ux-layout-section__item--table-view")
        items = item_specifics[0].find_all('div',class_= "ux-layout-section__row")
        
        specifics_dict = {}
        for i in items:
            a = i.find_all('div',class_ = "ux-labels-values__labels")
            b = i.find_all('div',class_ = "ux-labels-values__values-content") 
            for j in range(len(a)):
                specifics_dict[a[j].text.strip(":")] = b[j].text
        item_specs.append(specifics_dict)
    except:
        item_specs.append(None)

In [ ]:
new_data['condition'] = condition
new_data['price_usd'] = price_usd
new_data['item_specs'] = item_specs